# Abstract Matrices

In [1]:
import finite_algebras as alg
import numpy as np

import os
aa_path = os.path.join(os.getenv("PYPROJ"), "abstract_algebra")
alg_dir = os.path.join(aa_path, "Algebras")

# ex = alg.Examples(alg_dir)

In [2]:
ps3 = alg.generate_powerset_ring(3)
ps3.about()


** Ring **
Name: PSRing3
Instance ID: 4396867408
Description: Autogenerated Ring on powerset of {0, 1, 2} w/ symm. diff. (add) & intersection (mult)
Order: 8
Identity: {}
Commutative? Yes
Cyclic?: No
Elements:
   Index   Name   Inverse  Order
      0      {}      {}       1
      1     {0}     {0}       2
      2     {1}     {1}       2
      3     {2}     {2}       2
      4  {0, 1}  {0, 1}       2
      5  {0, 2}  {0, 2}       2
      6  {1, 2}  {1, 2}       2
      7 {0, 1, 2} {0, 1, 2}       2
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7],
 [1, 0, 4, 5, 2, 3, 7, 6],
 [2, 4, 0, 6, 1, 7, 3, 5],
 [3, 5, 6, 0, 7, 1, 2, 4],
 [4, 2, 1, 7, 0, 6, 5, 3],
 [5, 3, 7, 1, 6, 0, 4, 2],
 [6, 7, 3, 2, 5, 4, 0, 1],
 [7, 6, 5, 4, 3, 2, 1, 0]]
Mult. Identity: {0, 1, 2}
Mult. Commutative? Yes
Zero Divisors: ['{0}', '{1}', '{2}', '{0, 1}', '{0, 2}', '{1, 2}']
Multiplicative Cayley Table (showing indices):
[[0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 1, 1, 0, 1],
 [0, 0, 2, 0, 2, 0, 2, 2],
 [0,

In [3]:
ps3.zero

'{}'

In [4]:
class AbstractMatrix:

    def __init__(self, array, ring):
        if isinstance(array, np.ndarray):
            arr = array
        else:
            arr = np.array(array)
        self.__ring = ring
        self.__array = np.array(array, dtype='<U32')
    
    @classmethod
    def zeros(cls, shape, ring):
        arr = np.full(shape, ring.zero, dtype='<U32')
        return cls(arr, ring)
    
    @classmethod
    def random(cls, shape, ring):
        rand_indices = np.random.randint(ring.order, size=shape)
        rand_array = np.full(shape, ring.zero, dtype='<U32')
        for i in range(shape[0]):
            for j in range(shape[1]):
                rand_array[i, j] = ring.elements[rand_indices[i, j]]
        return cls(rand_array, ring)
    
    def get_array(self):
        return self.__array
    
    def get_shape(self):
        return self.__array.shape
    
    def get_nrows(self):
        return self.__array.shape[0]
    
    def get_ncols(self):
        return self.__array.shape[1]
    
    def get_algebra(self):
        return self.__ring
    
    def copy(self):
        return AbstractMatrix(np.copy(self.__array), self.__ring)
    
    def transpose(self):
        return AbstractMatrix(np.transpose(self.__array), self.__ring)
    
    def __mul__(self, other):  # Matrix multiplication using Ring operations
        # X * Y
        xarr =  self.__array
        yarr = other.__array
        xrows = self.get_nrows()
        xcols = self.get_ncols()
        yrows = other.get_nrows()
        ycols = other.get_ncols()
        product = None
        if xcols == yrows:
            if self.__ring == other.__ring:
                ring = self.__ring
                product = np.full((xrows, ycols), ring.zero, dtype='U32')
                for i in range(xrows):
                    for j in range(ycols):
                        for k in range(xcols):
                            product[i, j] = ring.add(product[i, j], ring.mult(xarr[i, k], yarr[k, j]))
            else:
                raise ValueError("The array algebras must be equal")
        else:
            raise ValueError(f"The array shapes are incompatible: {xcols} colums vs {yrows} rows")
        return AbstractMatrix(product, ring)
    
    def __add__(self, other):  # Matrix addition using Ring operations
        # X + Y
        xarr =  self.__array
        yarr = other.__array
        xshape = xarr.shape
        yshape = yarr.shape
        sum = None
        if xshape == yshape:
            if self.__ring == other.__ring:
                ring = self.__ring
                sum = np.full(xshape, ring.zero, dtype='U32')
                for i in range(xshape[0]):
                    for j in range(xshape[1]):
                        sum[i, j] = ring.add(xarr[i, j], yarr[i, j])
            else:
                raise ValueError("The array algebras must be equal")
        else:
            raise ValueError(f"The array shapes are not equal: {xshape} != {yshape}")
        return AbstractMatrix(sum, ring)
    
    def __sub__(self, other):  # Matrix subtraction using Ring operations
        # X - Y
        xarr =  self.__array
        yarr = other.__array
        xshape = xarr.shape
        yshape = yarr.shape
        sum = None
        if xshape == yshape:
            if self.__ring == other.__ring:
                ring = self.__ring
                sum = np.full(xshape, ring.zero, dtype='U32')
                for i in range(xshape[0]):
                    for j in range(xshape[1]):
                        sum[i, j] = ring.sub(xarr[i, j], yarr[i, j])
            else:
                raise ValueError("The array algebras must be equal")
        else:
            raise ValueError(f"The array shapes are not equal: {xshape} != {yshape}")
        return AbstractMatrix(sum, ring)

def determinant(array, ring):
    '''Compute the determinant of a square NumPy array of ring elements.'''
    nrows = array.shape[0]
    ncols = array.shape[1]
    if nrows != ncols:
        raise ValueError(f"Array must be square: ({nrows}, {ncols})")
    elif nrows == 1:
        return array[0, 0]
    elif nrows == 2:  # Recursion will stop here
        return ring.sub(ring.mult(array[0, 0], array[1, 1]), ring.mult(array[0, 1], array[1, 0]))
    else:
        # Use the Laplace expansion to recursively compute the determinant
        det = ring.zero  # The ring's "zero" element
        arr = np.delete(array, 0, 0)  # Copy array & delete first row
        for i in range(ncols):
            minor = np.delete(arr, i, 1)  # Copy arr & delete the ith column
            # Alternate "adding" and "subtracting", per the Laplace expansion
            if (-1)**i == 1:
                det = ring.add(det, ring.mult(array[0, i], determinant(minor, ring)))
            else:
                det = ring.sub(det, ring.mult(array[0, i], determinant(minor, ring)))
        return det

In [5]:
mat1 = AbstractMatrix.zeros((2, 3), ps3)

arr = mat1.get_array()
arr

array([['{}', '{}', '{}'],
       ['{}', '{}', '{}']], dtype='<U32')

In [6]:
fubar = [['{0}', '{1}', '{2}'],
         ['{2}', '{1}', '{0}']]

mat2 = AbstractMatrix(fubar, ps3)

mat2.get_array()

array([['{0}', '{1}', '{2}'],
       ['{2}', '{1}', '{0}']], dtype='<U32')

In [7]:
mat3 = mat2.transpose()
mat3.get_array()

array([['{0}', '{2}'],
       ['{1}', '{1}'],
       ['{2}', '{0}']], dtype='<U32')

In [8]:
mat2x3 = mat2 * mat3
mat2x3.get_array()

array([['{0, 1, 2}', '{1}'],
       ['{1}', '{0, 1, 2}']], dtype='<U32')

In [9]:
mat3x2 = mat3 * mat2
mat3x2.get_array()

array([['{0, 2}', '{}', '{}'],
       ['{}', '{}', '{}'],
       ['{}', '{}', '{0, 2}']], dtype='<U32')

In [10]:
try:
    mat2x2 = mat2 * mat2
    print(mat2x2.get_array())
except Exception as exc:
    print(exc)

The array shapes are incompatible: 3 colums vs 2 rows


In [11]:
mat2p2 = mat2 + mat2

mat2p2.get_array()

array([['{}', '{}', '{}'],
       ['{}', '{}', '{}']], dtype='<U32')

In [12]:
try:
    mat2p3 = mat2 + mat3
    print(mat2p3.get_array())
except Exception as exc:
    print(exc)

The array shapes are not equal: (2, 3) != (3, 2)


In [13]:
mat2m2 = mat2 - mat2
mat2.get_array()

array([['{0}', '{1}', '{2}'],
       ['{2}', '{1}', '{0}']], dtype='<U32')

## Numeric Determinant

Compute the determinant of a square numeric NumPy array:

## Abstract Determinant

The code below applies to a square abstract numpy array, not an abstract matrix.

### 2x2 Test

In [14]:
test2 = mat2x3.get_array()
test2

array([['{0, 1, 2}', '{1}'],
       ['{1}', '{0, 1, 2}']], dtype='<U32')

In [15]:
determinant(test2, mat2x3.get_algebra())

'{0, 2}'

In [16]:
ps3.mult('{0, 1, 2}', '{0, 1, 2}')

'{0, 1, 2}'

In [17]:
ps3.mult('{1}', '{1}')

'{1}'

In [18]:
ps3.sub('{0, 1, 2}', '{1}')

'{0, 2}'

### 3x3 Test

In [19]:
rnd1 = AbstractMatrix.random((3,3),ps3)
rnd1.get_array()

array([['{1, 2}', '{0, 1, 2}', '{0, 2}'],
       ['{0, 2}', '{}', '{1}'],
       ['{0}', '{1}', '{0, 1}']], dtype='<U32')

In [20]:
determinant(rnd1.get_array(), ps3)

'{0, 1}'

### 4x4 Test

In [21]:
rnd2 = AbstractMatrix.random((4,4),ps3)
rnd2.get_array()

array([['{0, 2}', '{1}', '{0}', '{2}'],
       ['{0}', '{0}', '{0}', '{0, 2}'],
       ['{0}', '{1}', '{1}', '{0, 1, 2}'],
       ['{2}', '{0, 2}', '{0, 2}', '{2}']], dtype='<U32')

In [22]:
determinant(rnd2.get_array(), ps3)

'{}'